# Building FF Scoring Model

In [8]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

# Data Loading

In [14]:
caption_path = "./caption_data"
relevant_columns = ["cap_feat", "img_feat", "mean"]
df_list = []
for pkl in os.listdir(caption_path):
    pkl_path = os.path.join(caption_path, pkl)
    img_df = pd.read_pickle(pkl_path)[relevant_columns]
    df_list.append(img_df)


In [17]:
data_df = pd.concat(df_list)
# print(data_df.head(5))
# print(len(data_df.index))

                                            cap_feat  \
0  [0.011870803, -0.006262237, -0.03857191, 0.050...   
1  [0.026949111, 0.007988956, -0.043752477, 0.063...   
2  [0.014411536, 0.003357767, -0.035420828, 0.022...   
3  [0.035097554, 0.045470487, -0.015568391, 0.067...   
4  [-0.024375904, 0.035241786, -0.022204943, 0.06...   

                                            img_feat      mean  
0  [[-0.05705367, -0.07593712, 0.01871802, 0.0108...  1.828421  
1  [[-0.05705367, -0.07593712, 0.01871802, 0.0108...  1.787506  
2  [[-0.05705367, -0.07593712, 0.01871802, 0.0108...  1.776693  
3  [[-0.05705367, -0.07593712, 0.01871802, 0.0108...  1.740367  
4  [[-0.05705367, -0.07593712, 0.01871802, 0.0108...  1.699167  
1981466


In [19]:
# Assuming df is your DataFrame

# Combine the embeddings into a single feature array
# Note: This assumes 'cap_feat' and 'img_feat' are each a list or array of 512 floats.
X = np.hstack([np.vstack(data_df['img_feat'].values), np.vstack(data_df['cap_feat'].values)])

# Your target variable
y = data_df['mean'].values/2

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
class HumorRatingNN(nn.Module):
    def __init__(self):
        super(HumorRatingNN, self).__init__()
        
        self.fc1 = nn.Linear(1024, 512) # 1024 inputs (512 from image + 512 from caption), to 512 outputs
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)  # 512 inputs to 256 outputs
        self.fc3 = nn.Linear(256, 1)    # 256 inputs to 1 output (your mean humor rating)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # No activation here, as we're predicting a continuous value
        return x

In [21]:
# Convert data to PyTorch tensors
train_data = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
test_data = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

# Create data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = HumorRatingNN()
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/Users/echen/miniforge3/envs/DVA/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.006202187389135361
